# Finding Arbitrage Opportunities in Cryptocurrencies

This notebook is an attempt to automate the process of finding arbitrage opportunities on various cryptocurrency exchanges for a number of currency pairs. This notebook uses the great work done here:
<https://github.com/ccxt/ccxt>

## NOTE:
This is an excercise in learning how to use python and ccxt. This notebook is **NOT** intended to be used for currecy trading.

In [1]:
import ccxt
import numpy as np

## Extract  exchanges
ccxt has been written to query multiple exchanges. The list of exchanges are found in ccxt.exchange. Dead exchanges are prefixed with "_" symbol before their names.

In [2]:
#get all ids from ccxt.exchange
ids = ccxt.exchanges
#remove ids with '_' at the start
ids=[id for id in ids if not '_' in id]
exchanges = {}
for id in ids:  # load all markets from all exchange exchanges
    exchange = getattr(ccxt, id)() # instantiate the exchange by id
    exchanges[id] = exchange # save it in a dictionary under its id for future use

## Find currency pairs in each exchange
'fetch_tickers' is used by ccxt to collect information about currency pairs from each exchange. However, 'fetch_tickers' doesn't work with all exchanges. The code below finds and stores only those exchanges for which 'fetch_tickers' works.

In [4]:
tickerS_ids=[] #initialise the list that contains all ids that are compatible with the 'fetch_tickers' command
nontickerS_ids=[] #initialise the list that contains all ids that are NOT compatible with the 'fetch_tickers' command
all_tickerS={} #initialise the dict that contains all tickers fetched by the 'fetch_tickers' command
for id in ids:
    try:
        tickers=(eval('ccxt.%s().fetch_tickers()' %id))
        tickerS_ids.append(id) #add the id that worked to 'tickerS_ids
        tickers={id:tickers} #put a key of the name "id" around the dict called tickers
        all_tickerS.update(tickers) #append the newly created dictionry to all_tickerS
    except: 
        nontickerS_ids.append(id)
        #print(id,'not compatible with tickers')
tickerS_ids[:5]

['acx', 'aofex', 'bequant', 'bibox', 'bigone']

The tickers that worked are stored in a file so that the code can be run from here in future if needed. Date and time are used so that there is no chance of overwriting any previous version of the files.

In [5]:
from datetime import datetime, timedelta
outF=open(datetime.now().strftime("%Y%m%d-%H-%M-%S_tickerS_ids.txt"),'w')
for line in tickerS_ids:
    outF.write(line)
    outF.write("\n")
outF.close()
outF = outF=open(datetime.now().strftime("%Y%m%d-%H-%M-%S_nontickerS_ids.txt"),'w')
for line in nontickerS_ids:
    outF.write(line)
    outF.write("\n")
outF.close()

## Create a list of lists with of currency pairs, exchange, ask and bid price
tickerS_ids is a dictionary with exchanges as keys and information about the exchange as the value. To find arbitrage, this code needs to:
1. find currency pairs which exist on two or more exchanges
2. have a bid and ask price
3. compare the bid and ask prices to check for arbitrage

So, a new list called 'bysymb' organises tickerS_ids by currency pair symbols.

In [6]:
#make a list of all symbols, exchanges, asks and bid prices. Manipulating a dictionary was too hard.
bysymb=[]
for ex in list(all_tickerS.keys()): #first level of keys is exchanges
    for s in list(all_tickerS[ex].keys()): #next level of keys is symbols
        ask=all_tickerS[ex][s]['ask']
        bid=all_tickerS[ex][s]['bid']
        bysymb.append([s,ex,ask,bid]) 
bysymb[:5]

[['BTC/AUD', 'acx', 13250.0, 10100.0],
 ['ETH/AUD', 'acx', 294.0, 200.01],
 ['ETC/AUD', 'acx', 18.88, 1.0],
 ['BCH/AUD', 'acx', 450.0, 0.01],
 ['LTC/AUD', 'acx', 139.0, 1.0]]

## Find and keep only those currency pairs which exist on more than one exchange
'artibrableSymbols' are those pairs that exist in two or more exchanges. 'indeces' are used to track those pairs.

In [7]:
#process the list to find all symbols, unique symbols, and arbitrable symbols
bysymb_array=np.array(bysymb) # make the list an array
allSymbols=list(bysymb_array[:,0]) #get the first column [:0] of the array which is all the symbols
uniqueSymbols=set(allSymbols) #get rid of duplicate symbols
arbitrableSymbols = sorted([symbol for symbol in uniqueSymbols if allSymbols.count(symbol) > 1]) #make arbitrableSymbols = symbol
#where symbol is allsymbol.count(symbol)>1 in uniquesymbols (in other words, the currency symbol must appear in more than one exchange)

# find the row numbers (indeces) corresponding to each arbitrablesymbol for further processing
indeces={} #initialise indeces as a dictionary
for s in arbitrableSymbols: #for each element of arbitrablesymbols...
    index=[] #initialise index as a list
    index=[i for i, x in enumerate(allSymbols) if x == s] #index = i if x in allsymbols is equal to s (the symbol in arbitrableSymbols)
    indeces.update({s:index}) #save the indeces of each symbol (s) in a dictonary.

## Calculate arbitrage opportunities
Finally, the bid prices for all exchanges that trade each currency pair are divided by the ask prices and the result checked to find any instances where the bid price is greater than the ask price (i.e. bid/ask > 1 or 1.1 to be safe). Instructions for each arbitrage opportunity found are captured in a list called 'tally'. 

In [8]:
#make a list "check" of all [s,ex,ask,bid] if not NONE and more than 0
tally=[]
arb_ex=[]
for s in list(indeces.keys()): #for each symbol s in indeces...
    ask=[]
    bid=[]
    asks=[]
    bids=[]
    index=[]
    check=[]
    for i in indeces[s]: #for each index in the list of indeces
        if bysymb_array[i,2] != None and bysymb_array[i,3] != None and bysymb_array[i,2] >0 and bysymb_array[i,3]>0:
            ask.append(bysymb_array[i,2]) #get the ith row and find the 2nd and 3rd elements (ask and bid prices)
            bid.append(bysymb_array[i,3])
            check.append(bysymb_array[i,:]) #keep this for checking the raw prices in bysymb_array
        else: ''
        #make a row matrix of asks and column matrix of bids for symbol and ex. Multiply bids with 1/asks
        asks=np.matrix(ask)               #convert asks and bids to matrices and transpose bids 
        bids=np.transpose(np.matrix(bid)) 
        mat=bids.dot(1/asks) #cross product of bids with 1/asks.
        #If asks < bids, abitrage may exist if bid/ask >1. To be safe, a score of 1.1 is used
        b = np.matrix(np.where(mat>1.1)) ### THIS FINDS THE ARBITRAGES
        n=0
        for n in range(0, np.shape(b)[1]):
            ask_ex=check[b[1,n]]
            bid_ex=check[b[0,n]]
            arb_ex.append(ask_ex[1])
            arb_ex.append(bid_ex[1])
            tally.append([s,'buy from', ask_ex[1], ask_ex[2], 'sell to', bid_ex[1],bid_ex[3],bid_ex[3]/ask_ex[2]])

In [9]:
tally[:5]

[['1ST/BTC',
  'buy from',
  'hitbtc',
  1.01252e-05,
  'sell to',
  'bittrex',
  1.127e-05,
  1.113064433295145],
 ['AAC/BTC',
  'buy from',
  'okex',
  2.05e-07,
  'sell to',
  'huobipro',
  9.261e-06,
  45.175609756097565],
 ['AAC/BTC',
  'buy from',
  'okex',
  2.05e-07,
  'sell to',
  'huobiru',
  9.261e-06,
  45.175609756097565],
 ['AAC/ETH',
  'buy from',
  'okex',
  8.34e-06,
  'sell to',
  'huobipro',
  0.00037223,
  44.63189448441247],
 ['ABBC/BTC',
  'buy from',
  'livecoin',
  1.77e-06,
  'sell to',
  'bitmart',
  1.33e-05,
  7.5141242937853105]]